In [13]:
!pip install langchain-fireworks
!pip install langchain | tail -n 1
!pip install wget | tail -n 1
!pip install sentence-transformers | tail -n 1
!pip install pydantic | tail -n 1
!pip install pymongo | tail -n 1
!pip install python-dotenv | tail -n 1
!pip install jq | tail -n 1
!pip install langchain-mongodb | tail -n 1

## Import Utility Packages

In [1]:
import os
import getpass
from dotenv import load_dotenv

## Environment Variables and Keys

### fireworks init

In [2]:
if "FIREWORKS_API_KEY" not in os.environ:
    os.environ["FIREWORKS_API_KEY"] = getpass.getpass("Please enter your fireworks.ai API KEY (hit enter): ")
    
fireworks_api_key = os.environ["FIREWORKS_API_KEY"]

Please enter your fireworks.ai API KEY (hit enter):  ········


In [20]:
query = "I want to introduce my daughter to science and spark her enthusiasm. What kind of gifts should I get her?"

In [136]:
from langchain_fireworks import Fireworks 

llm = Fireworks(
    fireworks_api_key=fireworks_api_key,
    model="accounts/fireworks/models/mistral-7b",
    temprature= 0.2,
    top_p=0.9,
    max_tokens=1024)
print(llm.invoke(query))

/Users/ashwin.gangadhar/anaconda3/lib/python3.9/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! temprature is not default parameter.
                temprature was transferred to model_kwargs.
                Please confirm that temprature is what you intended.
  warnings.warn(




Start young!

Kids are naturally curious, and are often more interested in learning about the world than they are about dolls or Barbies. Giving your kids an educational gift sends them a message that learning is fun and important. Here are some great science gift ideas for kids ages 1 to 5:

- Baby Einstein Discovery Stage Baby Toy
- Discovery Toys Magical Balance Blocks
- Fisher-Price Laugh and Learn Puppy
- Learning Resources Peek-A-Boo Science Explorations
- Leapfrog Imagicard Interactive Laptop
- Little People Workbench by Fisher Price
- Mega Bloks First Builders Alphabet Bus
- Mega Bloks First Builders Early Learning Set
- Melissa and Doug Giant Shape Sorting Cube
- Melissa and Doug Baby Touch-and-Feel-Growing with Colors
- Melissa and Doug Mix-it-Up Shape Sorter
- Melissa and Doug Sort and Serve Café
- Nuby Aquadoodle
- Num Chums ABC Blocks
- Num Chums 1,2,3 Counting Blocks
- Playskool Learning Tablet by VTech
- Play-Doh Mess-Free Super Shape Maker Set
- Pottery Barn Kids Bubb

In [216]:
from pymongo import MongoClient
if "MONGO_CONN" not in os.environ:
    os.environ["MONGO_CONN"] = getpass.getpass("Please enter your MongoDB Connection String (hit enter): ")
MONGO_CONN = os.environ["MONGO_CONN"]

## Load Documents

In [9]:
import wget
os.chdir("/Users/ashwin.gangadhar/projects/mbd-firework-rag")
filename = './amazon-products.jsonl'
url = "https://raw.githubusercontent.com/ashwin-gangadhar-mdb/mbd-watson-rag/main/amazon-products.jsonl?token=GHSAT0AAAAAACMUNLE34KEUDBF525Q55IUQZPPERKA"

In [11]:
if not os.path.isfile(filename):
    wget.download(url, out=filename)

In [14]:
from langchain.document_loaders import JSONLoader
loader = JSONLoader(file_path=filename, jq_schema=".text",text_content=False,json_lines=True)
docs = loader.load()

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm as notebook_tqdm
embeddings = HuggingFaceEmbeddings()

# Vector Store

We will be using an instance of [MongoDB Atlas Vector Search](https://www.mongodb.com/products/platform/atlas-vector-search), which is exposed as a [LangChain vector store](https://js.langchain.com/docs/modules/data_connection/vectorstores/integrations/mongodb_atlas).

Once you have added the document to the MongoDB collection you can then create a vector search index by following the steps provided in the [Atlas Search index tutorial](https://www.mongodb.com/docs/atlas/atlas-search/create-index/)

Use the following mapping json to create a vector search indexes for this tutorial

`{
  "mappings": {
    "dynamic": true,
    "fields": {
      "embedding": [
        {
          "dimensions": 768,
          "similarity": "cosine",
          "type": "knnVector"
        }
      ]
    }
  }
}`

In [217]:
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch
client = MongoClient(MONGO_CONN)
vcol = client["amazon"]["products"]
vectorstore = MongoDBAtlasVectorSearch(vcol, embeddings)

In [19]:
vectorstore.add_documents(docs)

[ObjectId('65ef26e993cfb56c73a6062a'),
 ObjectId('65ef26e993cfb56c73a6062b'),
 ObjectId('65ef26e993cfb56c73a6062c'),
 ObjectId('65ef26e993cfb56c73a6062d'),
 ObjectId('65ef26e993cfb56c73a6062e'),
 ObjectId('65ef26e993cfb56c73a6062f'),
 ObjectId('65ef26e993cfb56c73a60630'),
 ObjectId('65ef26e993cfb56c73a60631'),
 ObjectId('65ef26e993cfb56c73a60632'),
 ObjectId('65ef26e993cfb56c73a60633'),
 ObjectId('65ef26e993cfb56c73a60634'),
 ObjectId('65ef26e993cfb56c73a60635'),
 ObjectId('65ef26e993cfb56c73a60636'),
 ObjectId('65ef26e993cfb56c73a60637'),
 ObjectId('65ef26e993cfb56c73a60638'),
 ObjectId('65ef26e993cfb56c73a60639'),
 ObjectId('65ef26e993cfb56c73a6063a'),
 ObjectId('65ef26e993cfb56c73a6063b'),
 ObjectId('65ef26e993cfb56c73a6063c'),
 ObjectId('65ef26e993cfb56c73a6063d'),
 ObjectId('65ef26e993cfb56c73a6063e'),
 ObjectId('65ef26e993cfb56c73a6063f'),
 ObjectId('65ef26e993cfb56c73a60640'),
 ObjectId('65ef26e993cfb56c73a60641'),
 ObjectId('65ef26e993cfb56c73a60642'),
 ObjectId('65ef26e993cfb5

# Create Vector Search index

In [259]:
vcol.create_search_index({"definition":
        {"mappings":
         {"dynamic": True,
          "fields": {
            "embedding" : {
                "dimensions": 768,
                "similarity": "cosine",
                "type": "knnVector"
                }
            }}},
     "name": "default"
    })

'default'

In [277]:
list(vcol.list_search_indexes())


[{'id': '65ef43ba06ae8a6312e4061c',
  'name': 'default',
  'type': 'search',
  'status': 'READY',
  'queryable': True,
  'latestDefinitionVersion': {'version': 0,
   'createdAt': datetime.datetime(2024, 3, 11, 17, 47, 38, 254000)},
  'latestDefinition': {'mappings': {'dynamic': True,
    'fields': {'embedding': {'dimensions': 768,
      'similarity': 'cosine',
      'type': 'knnVector'}}}},
  'statusDetail': [{'hostname': 'atlas-12hrz1-shard-00-01',
    'status': 'READY',
    'queryable': True,
    'mainIndex': {'status': 'READY',
     'queryable': True,
     'definitionVersion': {'version': 0,
      'createdAt': datetime.datetime(2024, 3, 11, 17, 47, 38)},
     'definition': {'mappings': {'dynamic': True,
       'fields': {'embedding': {'type': 'knnVector',
         'dimensions': 768,
         'similarity': 'cosine'}}}}}}]}]

# Similiarity Search
<a id="similarity-search"></a>

We find the similar (i.e. relevant) texts to our query.  You can modify the number of results returned with `k` parameter in the `similarity_search` method below.

In [280]:
texts_sim = vectorstore.similarity_search(query, k=10)
print("First 100 characters of relevant texts.")
for i in range(len(texts_sim)):
        print("Text " + str(i) + ": " + str(texts_sim[i].page_content[0:100]))

First 100 characters of relevant texts.
Text 0: Product Name: Scientific Explorer My First Science Kids Science Experiment Kit

About Product: Exper
Text 1: Product Name: Learning Resources Primary Science Deluxe Lab Set, Science Kit, 45 Piece Set, Ages 3+

Text 2: Product Name: Scientific Explorer Mind Blowing Science Kit with Scientific Explorer Magic Science fo
Text 3: Product Name: The Young Scientists Club Science Art Fusion Rainbows Kit

About Product: Great hands 
Text 4: Product Name: Learning Resources STEM Magnets Activity Set, Science Lessons, 24 Pieces, Ages 5+

Abo
Text 5: Product Name: Hey! Play! Kids Science Kit-Lab Set to Create Solutions, Litmus Paper, & More-Great Fu
Text 6: Product Name: Be Amazing! Toys Universe Unboxed by Scishow: Survival Science Lab

About Product: Be 
Text 7: Product Name: Be Amazing! Toys Get Slimed! Science Kit

About Product: Explore the world of polymers
Text 8: Product Name: Scientific Explorer Glow in The Dark Fun Lab Kids Science Experime

# RAG Generation (Explicit Context Control)
<a id="rag-explicit"></a>

RAG generation using our [model](#language-model) and explicit relevant knowledge (documents) from our [similarity search](#similarity-search).

In [284]:
from langchain.chains import RetrievalQA,LLMChain
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(template="""
### Persona: You are e-commerce product recommendation bot. Provided the user query, undertand the purchase intent and recommend relevant products
### User Query: {question},
### Product search response: {context},
### Product Recommendation:

""",input_variables=["question", "context"])
# combine_documents_chain = LLMChain(llm=llm,prompt=prompt)
retriever = vectorstore.as_retriever(search_kwargs={"k":10}) # get top 10 documents

In [285]:
chain_type_kwargs = {"prompt": prompt}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",\
                                  retriever=retriever,\
                                  chain_type_kwargs=chain_type_kwargs)

In [287]:
result = chain.invoke(query)
print(result['answer'])

 1. Scientific Explorer My First Science Kids Science Experiment Kit
2. Learning Resources Primary Science Deluxe Lab Set, Science Kit, 45 Piece Set, Ages 3+
3. Scientific Explorer Mind Blowing Science Kit with Scientific Explorer Magic Science for Wizards Only Kit Bundle
4. The Young Scientists Club Science Art Fusion Rainbows Kit
5. Learning Resources STEM Magnets Activity Set, Science Lessons, 24 Pieces, Ages 5+
6. Hey! Play! Kids Science Kit-Lab Set to Create Solutions, Litmus Paper, & More-Great Fun & Educational Stem Learning Activity for Boys & Girls
7. Be Amazing! Toys Universe Unboxed by Scishow: Survival Science Lab
8. Be Amazing! Toys Get Slimed! Science Kit
9. Scientific Explorer Glow in The Dark Fun Lab Kids Science Experiment Kit
10. Educational Insights GeoSafari Rockin' Reactions Chemistry Set

